 # <font color = #254117>[EEP 147]: ESG Analysis Round 1</font>

<div style="width:image width px; font-size:80%; text-align:center;"><img src="big_creek.jpg" alt="alternate text" width="500" height="height" style="padding-bottom:0.5em;" />Big Creek Hydroelectric Project - Southern California Edison</div>

This notebook can be utilized for analysis of the Electricity Strategy Game. This notebook builds on the Round 0 notebook by allowing carbon prices to be adjusted in more of the function, including separately for each of Rounds 4, 5, and 6 in the NPV of expected profits function.

First on our agenda is to import **<font color = ##008700>dependencies</font>** -- packages in Python that add to the basic functions in Python -- same as before.

In [ ]:
from datascience import *
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline
import numpy as np
import pandas as pd
from ipywidgets import interact, interactive, Dropdown, IntSlider
import ipywidgets as widgets
from functools import partial
plt.style.use('fivethirtyeight')
plt.rcParams["figure.figsize"] = [10,6]

Next, let's select your portfolio. In the cell below, assign **section** to the section number that corresponds to your own according to the following table.

| Number | Section Time  |
|---------|---------------|
| 1 | Wednesday 8am |
| 2 | Wednesday 9am |
| 3 | Friday 2pm |
| 4 | Friday 3pm |

Assign **YOUR_PORTFOLIO** to the name of your portfolio from the following choices:

**'Bay Views',
 'Beachfront',
 'Big Coal',
 'Big Gas',
 'East Bay',
 'Fossil Light',
 'Old Timers'**
 
The variable **periods_completed** should be set to the most recently concluded round.

The variable **period** should be set to the round that you want to analyze.

The variable **pab_periods** should contain each of the periods for which there was or will be a pay-as-bid auction.  This shouldn't change.

In [ ]:
section = 1
YOUR_PORTFOLIO = "Big Coal"
periods_completed = 0
period = 0
pab_periods = [1]

Next we import the demand forecasts.

In [ ]:
demand_table = Table.read_table('demand.csv')

Next we import our Bids and Porfolios tables, with all bids set to MC.

In [ ]:
bids_mc = Table.read_table('MC_bids.csv').sort('PORTFOLIO')
ESG = Table.read_table('ESGPorfolios.csv')

In the following cell we will join the tables based on the column **Plant_ID**.

In [ ]:
joined_table_all = bids_mc.join("PLANT_ID", ESG, "Plant_ID").sort("PLANT_ID")

Now, let's create a function to determine the price given the bids and demand.

In [ ]:
def price_calc(input_table, demand, hour, period):
    #hour and period determine which bids are taken from joined_table
    joined_table = input_table.copy()
    sorted_table = joined_table.where("PERIOD", period).sort("PRICE" + str(hour), descending = False)
    price = 0
    sum_cap = 0
    for i in range(0,len(sorted_table['Capacity_MW'])):
        if sum_cap + sorted_table['Capacity_MW'][i] > demand:
            price = sorted_table['PRICE' + str(hour)][i]
            break
        else:
            sum_cap += sorted_table['Capacity_MW'][i]
            price = sorted_table['PRICE' + str(hour)][i]
    return price

Now, let's create a function to plot the market in a given hour and period.

In [ ]:
def find_x_pos(widths):
    cumulative_widths = [0]
    cumulative_widths.extend(np.cumsum(widths))
    half_widths = [i/2 for i in widths]
    x_pos = []
    for i in range(0, len(half_widths)):
        x_pos.append(half_widths[i] + cumulative_widths[i])
    return x_pos

def price_line_plot(price):
    plt.axhline(y=price, color='r', linewidth = 2)
    print("Price: " + str(price))
    
def demand_plot(demand):
    plt.axvline(x=demand, color='r', linewidth = 2)
    print("Demand: " + str(demand))
    
def adjust_bids_by_cp(input_table, hour, period, carbon_price):
    joined_table = input_table.copy()
    joined_table["Total_Var_Cost_USDperMWH"] += carbon_price * joined_table["Carbon_tonsperMWH"]
    joined_table["PRICE" + str(hour)] += carbon_price * joined_table["Carbon_tonsperMWH"]
    return(joined_table)
    
def user_defined_bids(input_table, hour, period, my_portfolio, def_my_bids, def_others_bids):
    joined_table = input_table.copy()
    joined_df = joined_table.to_df()
    if def_my_bids:
        joined_df.loc[(joined_df["Group"] == my_portfolio) & 
                      (joined_df["PERIOD"] == period), 
                      "PRICE" + str(hour)] =  list(globals()["bids_" + my_portfolio.replace(" ", "").lower()].values())
    if def_others_bids:
        for group in set(joined_table['Group']):
            if group != my_portfolio:
                joined_df.loc[(joined_df["Group"] == group) & 
                              (joined_df["PERIOD"] == period), 
                              "PRICE" + str(hour)] =  list(globals()["bids_" + group.replace(" ", "").lower()].values())
    joined_table = Table.from_df(joined_df)
    return(joined_table)

def market_plot(input_table, hour, period, carbon_price, my_portfolio, def_my_bids, def_others_bids):
    demand = demand_table.where("round", period).where("hour", hour)["load"].item()
    joined_table = adjust_bids_by_cp(input_table, hour, period, carbon_price)
    joined_table = user_defined_bids(joined_table, hour, period, my_portfolio, def_my_bids, def_others_bids)
    
    sorted_joined_table = joined_table.where("PERIOD", period).sort("PRICE" + str(hour), descending = False)
    width = sorted_joined_table.column("Capacity_MW")
    height = sorted_joined_table.column('PRICE' + str(hour))
    x_vals = find_x_pos(width)
    energy_colors_dict = {}
    count = 0
    colors = ['#EC5F67', '#F29056', '#F9C863', '#99C794', '#5FB3B3', '#6699CC', '#C594C5']
    for i in set(joined_table['Group']):
        energy_colors_dict[i] = colors[count]
        count += 1
    colors_mapped = list(pd.Series(sorted_joined_table['Group']).map(energy_colors_dict))
    sorted_joined_table = sorted_joined_table.with_column('Color', colors_mapped)
    group_colors = sorted_joined_table.group("Group", lambda x: x).select("Group", "Color")
    group_colors["Color"] = group_colors.apply(lambda x: x[0], "Color")
    price = price_calc(sorted_joined_table, demand, hour, period)
    # Make the plot
    plt.figure(figsize=(9,6))
    plt.bar(x_vals, height, width=width, color=sorted_joined_table['Color'], edgecolor = "black")
    patches = []
    for row in group_colors.rows:
        patches += [mpatches.Patch(color=row.item("Color"), label=row.item("Group"))]
    plt.legend(handles=patches, bbox_to_anchor=(1.1,1))
    plt.title('Energy Market')
    plt.xlabel('Capacity_MW')
    plt.ylabel('Price')
    price_line_plot(price)
    demand_plot(demand)

In the next cell, we can define the bids.

In [ ]:
bids_bigcoal = {'fourcorners' : 36.5, 'alamitos7' : 73.72, 'huntingtonbeach1_2' : 40.5, 'huntingtonbeach5' : 66.5, 
                         'redondo5_6' : 41.94, 'redondo7_8' : 41.94}

bids_biggas = {'elsegundo1_2' : 44.83, 'elsegundo3_4' : 41.22, 'longbeach' : 52.5, 'northisland' : 65.5, 
               'encina' : 41.67, 'kearny' : 90.06, 'southbay' : 43.83}

bids_bayviews = {'morrobay1_2' : 38.78, 'morrobay3_4' : 36.61, 'mosslanding6' : 32.56, 'mosslanding7' : 32.56, 
                 'oakland' : 61.17}

bids_beachfront = {'coolwater' : 42.39, 'etiwanda1_4' : 42.67, 'etiwanda5' : 62.89, 'ellwood' : 75.61, 'mandalay1_2' : 39.06,
                   'mandalay3' : 52.06, 'ormondbeach1' : 38.06, 'ormondbeach2' : 38.06}

bids_eastbay = {'pittsburgh1_4' : 40.94, 'pittsburgh5_6' : 36.61, 'pittsburgh7' : 59.72, 'contracosta4_5' : 58.28, 
                         'contracosta6_7' : 39.5, 'potrerohill' : 69.83}

bids_oldtimers = {'bigcreek' : 0, 'mohave1' : 34.5, 'mohave2' : 34.5, 'highgrove' : 49.61, 'sanbernadino' : 53.94}

bids_fossillight = {'humboldt' : 47.44, 'helms' : 0.5, 'hunterspoint1_2' : 49.17, 'hunterspoint4' : 75.89, 
                    'diablocanyon1' : 11.5}

The next cell plots the market.

In [ ]:
interact(lambda hour, period, my_portfolio, def_my_bids, def_others_bids:
         market_plot(input_table = joined_table_all,
                     hour = hour,
                     period = period,
                     carbon_price = 0,
                     my_portfolio = my_portfolio,
                     def_my_bids = def_my_bids,
                     def_others_bids = def_others_bids),
         hour = Dropdown(options=list(range(1,5))), 
         period = Dropdown(options=list(range(1,7))),
         my_portfolio = Dropdown(options=np.unique(joined_table_all["Group"])),
         def_my_bids = Dropdown(options=[False, True]),
         def_others_bids = Dropdown(options=[False, True]))

Let's zoom in on individual portolios to see which of their plants are operating.  A plant which bids a price below the market-clearing price will operate.  First, let's define a plotting function.

In [ ]:
def portfolio_plot(input_table, hour, period, carbon_price, my_portfolio, def_my_bids, def_others_bids):
    demand = demand_table.where("round", period).where("hour", hour)["load"].item()
    joined_table = adjust_bids_by_cp(input_table, hour, period, carbon_price)
    joined_table = user_defined_bids(joined_table, hour, period, my_portfolio, def_my_bids, def_others_bids)
    
    sorted_joined_table = joined_table.where("PERIOD", period).sort("PRICE" + str(hour), descending = False)
    your_source = sorted_joined_table.where("Group", my_portfolio)
    width_yours = your_source.column("Capacity_MW")
    height_yours = your_source.column('PRICE' + str(hour))
    height_yours_marginal_cost = your_source.column("Total_Var_Cost_USDperMWH")
    new_x_yours = find_x_pos(width_yours)
    label_yours = your_source.column("PLANT")
    energy_colors_dict = {}
    count = 0
    colors = ['#EC5F67', '#F29056', '#F9C863', '#99C794', '#5FB3B3', '#6699CC', '#C594C5']
    for i in set(joined_table['Group']):
        energy_colors_dict[i] = colors[count]
        count += 1
    colors_mapped = list(pd.Series(sorted_joined_table['Group']).map(energy_colors_dict))
    sorted_joined_table = sorted_joined_table.with_column('Color', colors_mapped)
    group_colors = sorted_joined_table.group("Group", lambda x: x).select("Group", "Color")
    group_colors["Color"] = group_colors.apply(lambda x: x[0], "Color")
    price = price_calc(sorted_joined_table, demand, hour, period)
    # Make the plot
    plt.figure(figsize=(9,6))
    plt.bar(new_x_yours, height_yours, width=width_yours, 
            color = energy_colors_dict[my_portfolio], edgecolor = "black")
    plt.title("Bids: " + my_portfolio)
    plt.xlabel('Capacity_MW')
    plt.ylabel('Price')
    for new_x_i, height_i, label_i in zip(new_x_yours, height_yours, label_yours):
        plt.text(new_x_i, height_i, label_i, ha='center', va='bottom', fontsize=8)
    price_line_plot(price)
    plt.show()

Below we call our plotting function, allowing for the portfolio to be defined through an interactive dropdown box.  Remember that we are back to looking at the market without setting a carbon price.

In [ ]:
interact(lambda hour, period, my_portfolio, def_my_bids, def_others_bids:
         portfolio_plot(input_table = joined_table_all,
                        hour = hour,
                        period = period,
                        carbon_price = 0,
                        my_portfolio = my_portfolio,
                        def_my_bids = def_my_bids,
                        def_others_bids = def_others_bids),
         hour = Dropdown(options=list(range(1,5))), 
         period = Dropdown(options=list(range(1,7))),
         my_portfolio = Dropdown(options=np.unique(joined_table_all["Group"])),
         def_my_bids = Dropdown(options=[False, True]),
         def_others_bids = Dropdown(options=[False, True]))

The following function will allow us to zoom in further on only the operating plants.  Capacity is automatically reduced to supplied capacity for marginal plants.

In [ ]:
def marginal_cost_plot(input_table, hour, period, carbon_price, my_portfolio, def_my_bids, def_others_bids):
    demand = demand_table.where("round", period).where("hour", hour)["load"].item()
    joined_table = adjust_bids_by_cp(input_table, hour, period, carbon_price)
    joined_table = user_defined_bids(joined_table, hour, period, my_portfolio, def_my_bids, def_others_bids)
    
    sorted_joined_table = joined_table.where("PERIOD", period).sort("PRICE" + str(hour), descending = False)
    your_source = sorted_joined_table.where("Group", my_portfolio)
    price = price_calc(sorted_joined_table, demand, hour, period)
    
    nonmarg_capacity = sum(sorted_joined_table.where('PRICE' + str(hour), are.below(price))["Capacity_MW"])
    marg_capacity = sum(sorted_joined_table.where('PRICE' + str(hour), are.equal_to(price))["Capacity_MW"])
    marg_demand = demand - nonmarg_capacity
    marg_proportion = marg_demand / marg_capacity

    height_yours = your_source.column('PRICE' + str(hour))
    marginal_plants = np.where(height_yours == price)[0]
    width_yours = your_source.column("Capacity_MW")
    width_yours[marginal_plants] = width_yours[marginal_plants] * marg_proportion
    new_x_yours = find_x_pos(width_yours)

    height_yours_marginal_cost = your_source.column("Total_Var_Cost_USDperMWH")
    label_yours = your_source.column("PLANT")
    energy_colors_dict = {}
    count = 0
    colors = ['#EC5F67', '#F29056', '#F9C863', '#99C794', '#5FB3B3', '#6699CC', '#C594C5']
    for i in set(joined_table['Group']):
        energy_colors_dict[i] = colors[count]
        count += 1
    colors_mapped = list(pd.Series(sorted_joined_table['Group']).map(energy_colors_dict))
    sorted_joined_table = sorted_joined_table.with_column('Color', colors_mapped)
    group_colors = sorted_joined_table.group("Group", lambda x: x).select("Group", "Color")
    group_colors["Color"] = group_colors.apply(lambda x: x[0], "Color")
    plt.figure(figsize=(9,6))
    num_x = len(your_source.where('PRICE' + str(hour), are.below_or_equal_to(price))[1])
    plt.bar(new_x_yours[:num_x], height_yours_marginal_cost[:num_x], width=width_yours[:num_x], 
            color = energy_colors_dict[my_portfolio],
            edgecolor = "black")
    plt.title("Marginal Cost: " + my_portfolio)
    plt.xlabel('Capacity_MW')
    plt.ylabel('Marginal Cost')
    for new_x_i, height_i, label_i in zip(new_x_yours[:num_x], height_yours_marginal_cost[:num_x], label_yours[:num_x]):
        plt.text(new_x_i, height_i, label_i, ha='center', va='bottom', fontsize=8)
    price_line_plot(price)
    plt.show()

In the following plot, the empty area in between the price line and the tops of the bars (the marginal cost) is the profit.

In [ ]:
interact(lambda hour, period, my_portfolio, def_my_bids, def_others_bids:
         marginal_cost_plot(input_table = joined_table_all,
                            hour = hour,
                            period = period,
                            carbon_price = 0,
                            my_portfolio = my_portfolio,
                            def_my_bids = def_my_bids,
                            def_others_bids = def_others_bids),
         hour = Dropdown(options=list(range(1,5))), 
         period = Dropdown(options=list(range(1,7))),
         my_portfolio = Dropdown(options=np.unique(joined_table_all["Group"])),
         def_my_bids = Dropdown(options=[False, True]),
         def_others_bids = Dropdown(options=[False, True]))

Now we can finally calculate profit. The function below will find it.  Ignore the profit_pab() function for now (it will be explained more in Round 1).

In [ ]:
def profit(input_table, hour, period, carbon_price, my_portfolio, def_my_bids, def_others_bids):
    if period in pab_periods:
        return(profit_pab(input_table, hour, period, carbon_price, my_portfolio, def_my_bids, def_others_bids))
        
    demand = demand_table.where("round", period).where("hour", hour)["load"].item()
    joined_table = adjust_bids_by_cp(input_table, hour, period, carbon_price)
    joined_table = user_defined_bids(joined_table, hour, period, my_portfolio, def_my_bids, def_others_bids)
    
    sorted_joined_table = joined_table.where("PERIOD", period).sort("PRICE" + str(hour), descending = False)
    price = price_calc(sorted_joined_table, demand, hour, period)
    
    nonmarg_capacity = sum(sorted_joined_table.where('PRICE' + str(hour), are.below(price))["Capacity_MW"])
    marg_capacity = sum(sorted_joined_table.where('PRICE' + str(hour), are.equal_to(price))["Capacity_MW"])
    marg_demand = demand - nonmarg_capacity
    marg_proportion = marg_demand / marg_capacity
    
    sorted_table = sorted_joined_table.where("Group", my_portfolio)
    capacity_subset = sum(sorted_table.where('PRICE' + str(hour), are.below(price))["Capacity_MW"])
    capacity_subset += sum(sorted_table.where('PRICE' + str(hour), are.equal_to(price))["Capacity_MW"] * marg_proportion)
    revenue = capacity_subset * price
    cost = 0
    for i in range(len(sorted_table.where('PRICE' + str(hour), are.below(price))["Total_Var_Cost_USDperMWH"])):
        cost += sorted_table.where('PRICE' + str(hour), are.below(price))["Total_Var_Cost_USDperMWH"][i]\
        * sorted_table.where('PRICE' + str(hour), are.below(price))["Capacity_MW"][i]
    for i in range(len(sorted_table.where('PRICE' + str(hour), are.equal_to(price))["Total_Var_Cost_USDperMWH"])):
        cost += sorted_table.where('PRICE' + str(hour), are.equal_to(price))["Total_Var_Cost_USDperMWH"][i]\
        * (sorted_table.where('PRICE' + str(hour), are.equal_to(price))["Capacity_MW"][i] * marg_proportion)
    return revenue - cost

def profit_pab(input_table, hour, period, carbon_price, my_portfolio, def_my_bids, def_others_bids):
    demand = demand_table.where("round", period).where("hour", hour)["load"].item()
    joined_table = adjust_bids_by_cp(input_table, hour, period, carbon_price)
    joined_table = user_defined_bids(joined_table, hour, period, my_portfolio, def_my_bids, def_others_bids)
    
    sorted_joined_table = joined_table.where("PERIOD", period).sort("PRICE" + str(hour), descending = False)
    price = price_calc(sorted_joined_table, demand, hour, period)
    
    nonmarg_capacity = sum(sorted_joined_table.where('PRICE' + str(hour), are.below(price))["Capacity_MW"])
    marg_capacity = sum(sorted_joined_table.where('PRICE' + str(hour), are.equal_to(price))["Capacity_MW"])
    marg_demand = demand - nonmarg_capacity
    marg_proportion = marg_demand / marg_capacity
    
    sorted_table = sorted_joined_table.where("Group", my_portfolio)
    revenue = 0
    for i in range(len(sorted_table.where('PRICE' + str(hour), are.below(price))['PRICE' + str(hour)])):
        revenue += sorted_table.where('PRICE' + str(hour), are.below(price))['PRICE' + str(hour)][i]\
        * sorted_table.where('PRICE' + str(hour), are.below(price))["Capacity_MW"][i]
    for i in range(len(sorted_table.where('PRICE' + str(hour), are.equal_to(price))['PRICE' + str(hour)])):
        revenue += sorted_table.where('PRICE' + str(hour), are.equal_to(price))['PRICE' + str(hour)][i]\
        * (sorted_table.where('PRICE' + str(hour), are.equal_to(price))["Capacity_MW"][i] * marg_proportion)
    cost = 0
    for i in range(len(sorted_table.where('PRICE' + str(hour), are.below(price))["Total_Var_Cost_USDperMWH"])):
        cost += sorted_table.where('PRICE' + str(hour), are.below(price))["Total_Var_Cost_USDperMWH"][i]\
        * sorted_table.where('PRICE' + str(hour), are.below(price))["Capacity_MW"][i]
    for i in range(len(sorted_table.where('PRICE' + str(hour), are.equal_to(price))["Total_Var_Cost_USDperMWH"])):
        cost += sorted_table.where('PRICE' + str(hour), are.equal_to(price))["Total_Var_Cost_USDperMWH"][i]\
        * (sorted_table.where('PRICE' + str(hour), are.equal_to(price))["Capacity_MW"][i] * marg_proportion)
    return revenue - cost

In [ ]:
interact(lambda hour, period, my_portfolio, def_my_bids, def_others_bids:
         print("Profit: $" + str(round(
             profit(input_table = joined_table_all,
                    hour = hour,
                    period = period,
                    carbon_price = 0,
                    my_portfolio = my_portfolio,
                    def_my_bids = def_my_bids,
                    def_others_bids = def_others_bids),
             2))),
         hour = Dropdown(options=list(range(1,5))), 
         period = Dropdown(options=list(range(1,7))),
         my_portfolio = Dropdown(options=np.unique(joined_table_all["Group"])),
         def_my_bids = Dropdown(options=[False, True]),
         def_others_bids = Dropdown(options=[False, True]))

We can now calculate the amount of emissions in the current hour.

In [ ]:
def emissions(input_table, hour, period, carbon_price, my_portfolio, def_my_bids, def_others_bids):
    demand = demand_table.where("round", period).where("hour", hour)["load"].item()
    joined_table = adjust_bids_by_cp(input_table, hour, period, carbon_price)
    joined_table = user_defined_bids(joined_table, hour, period, my_portfolio, def_my_bids, def_others_bids)
    
    nonmarg_capacity = sum(sorted_joined_table.where('PRICE' + str(hour), are.below(price))["Capacity_MW"])
    marg_capacity = sum(sorted_joined_table.where('PRICE' + str(hour), are.equal_to(price))["Capacity_MW"])
    marg_demand = demand - nonmarg_capacity
    marg_proportion = marg_demand / marg_capacity
    
    sorted_table = sorted_joined_table.where("Group", my_portfolio)
    emissions = 0
    for i in range(len(sorted_table.where('PRICE' + str(hour), are.below(price))["Total_Var_Cost_USDperMWH"])):
        emissions += sorted_table.where('PRICE' + str(hour), are.below(price))["Carbon_tonsperMWH"][i]\
        * sorted_table.where('PRICE' + str(hour), are.below(price))["Capacity_MW"][i]
    for i in range(len(sorted_table.where('PRICE' + str(hour), are.equal_to(price))["Total_Var_Cost_USDperMWH"])):
        emissions += sorted_table.where('PRICE' + str(hour), are.equal_to(price))["Carbon_tonsperMWH"][i]\
        * (sorted_table.where('PRICE' + str(hour), are.equal_to(price))["Capacity_MW"][i] * marg_proportion)
    return emissions

In [ ]:
interact(lambda hour, period, my_portfolio, def_my_bids, def_others_bids:
         print("Emissions: " + str(round(
             profit(input_table = joined_table_all,
                    hour = hour,
                    period = period,
                    carbon_price = 0,
                    my_portfolio = my_portfolio,
                    def_my_bids = def_my_bids,
                    def_others_bids = def_others_bids),
             2)) + " Tons CO2"),
         hour = Dropdown(options=list(range(1,5))), 
         period = Dropdown(options=list(range(1,7))),
         my_portfolio = Dropdown(options=np.unique(joined_table_all["Group"])),
         def_my_bids = Dropdown(options=[False, True]),
         def_others_bids = Dropdown(options=[False, True]))

Thanks for help from: Alec Kan, Alma Pineda, Aarish Irfan, Elaine Chien, Octavian Sima, and Eric Van Dusen.